<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
    <font color='red'>To begin: Click anywhere in this cell and press <kbd>Run</kbd> on the menu bar. This executes the current cell and then highlights the next cell. There are two types of cells. A <i>text cell</i> and a <i>code cell</i>. When you <kbd>Run</kbd> a text cell (<i>we are in a text cell now</i>), you advance to the next cell without executing any code. When you <kbd>Run</kbd> a code cell (<i>identified by <span style="font-family: courier; color:black; background-color:white;">In[ ]:</span> to the left of the cell</i>) you advance to the next cell after executing all the Python code within that cell. Any visual results produced by the code (text/figures) are reported directly below that cell. Press <kbd>Run</kbd> again. Repeat this process until the end of the notebook. <b>NOTE:</b> All the cells in this notebook can be automatically executed sequentially by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart and Run All</kbd>. Should anything crash then restart the Jupyter Kernal by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart</kbd>, and start again from the top.
        
</div>

### 1. Import Packages

In [2]:
import numpy as np
import pandas as pd
import cimcb as cb

print('All packages successfully loaded')

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


All packages successfully loaded


### 2. Load Data & Peak Sheet

In [3]:
home = 'data/'
file = 'MTBLS90.xlsx'

DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak')

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 968 TOTAL PEAKS: 189
Done!


### 3. Extract X & Y

In [4]:
# Select Subset of Data
DataTable2 = DataTable[(DataTable.Class == 1) | (DataTable.Class == 0)]

# Create a Binary Y Vector 
Outcomes = DataTable2['Class']
Y = Outcomes.values 

# Extract and Scale Metabolite Data 
peaklist = PeakTable['Name']                           
XT = DataTable2[peaklist]
XTlog = np.log(XT)                                          
XTscale = cb.utils.scale(XTlog, method='auto')              
XTknn = cb.utils.knnimpute(XTscale, k=3)  

### 4. Hyperparameter Optimisation

In [ ]:
# Parameter Dictionary
lr = [0.001,0.005,0.01,0.02,0.03,0.1,0.5,1]

param_dict = dict(learning_rate=lr,
                  n_neurons=2,
                  epochs=1000,
                  momentum=0.5,
                  decay=0,
                  loss='binary_crossentropy')

# Initialise
cv = cb.cross_val.kfold(model=cb.model.NN_SigmoidSigmoid,
                                X=XTknn,
                                Y=Y,
                                param_dict=param_dict,
                                folds=5,
                                n_mc=10)

# Run and Plot
cv.run()
cv.plot(metric='auc')
cv.plot(metric='r2q2')

In [ ]:
# To do:
# Give option to select which LV to plot against rather than all
# Size option for plots
# Scatter option: plot Full, CV, FULL&CV or None
# Legend option for plots
# Give option to select learning rate or epochs etc..

# cv.plot_projections()

In [ ]:
# To do:
# Parallel

model = cb.model.NN_SigmoidSigmoid(learning_rate=0.02,
                                      n_neurons=2,
                                      epochs=1000,
                                      momentum=0.5,
                                      decay=0,
                                      loss='binary_crossentropy')
model.train(XTknn, Y)
model.test(XTknn)
model.permutation_test(nperm=100)

### 5. Build Model & Evaluate

In [6]:
# Build Model
model = cb.model.NN_SigmoidSigmoid(learning_rate=0.02,
                                      n_neurons=2,
                                      epochs=1000,
                                      momentum=0.5,
                                      decay=0,
                                      loss='binary_crossentropy')
model.train(XTknn, Y)
model.test(XTknn)

# Evaluate Model 
model.evaluate(cutoffscore=0.5) 

Loading BokehJS ...

### 6. Visualise

In [8]:
# To do:
    # Allow the option for Perc or BCA

bootmodel = cb.bootstrap.BC(model, bootnum=100)
bootmodel.run()

Bootstrap Resample: 100%|██████████| 100/100 [14:43<00:00, 12.29s/it]


In [9]:
# To do: align midline properly
# Size option for plots
# Scatter option: plot Full, CV, FULL&CV or None, Inner circles
# Legend option for plots

bootmodel.plot_projections()

Loading BokehJS ...

In [10]:
bootmodel.plot_loadings(PeakTable,
                        peaklist,
                        ylabel='Label',  # change ylabel to 'Name' 
                        sort=True)      # change sort to False

Loading BokehJS ...

In [11]:
peakSheet_featureimportance = bootmodel.plot_featureimportance(PeakTable,
                                         peaklist,
                                         ylabel='Label',  # change ylabel to 'Name' 
                                         sort=True,
                                         sort_ci=True)      # change sort to False

Loading BokehJS ...

### 7. Evaluate

In [13]:
#### Testing a few options: ####
# parametric = True / False / None / 1
# True: Upper limit exactly mirrors lower limit
# False: Same as True, except if Upper CI was already 1, upper limit remains 1 at given point
# None: Upper limit is always 1
# 1: No upper limit

# BC = True / False; whether to do bias-correction for ROC Curve

bootmodel.evaluate(parametric=None, bc=True) # Options: parametric = True / False 

Loading BokehJS ...